Imports

In [ ]:
!pip uninstall xgboost
!pip uninstall missingpy
!pip install xgboost==2.0.0
!pip install scikeras==0.12.0
!pip install missingpy==0.2.0
!pip install statsmodels

In [ ]:
import os
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from scipy.stats import norm


from pandas.compat import numpy
import scipy

from sklearn import ensemble
from sklearn.linear_model import LinearRegression


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


from scikeras.wrappers import KerasClassifier, KerasRegressor



from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import sklearn.metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import export_text

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import collections

import time

import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
import os
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from scipy.stats import norm

from sklearn import ensemble
from sklearn.linear_model import LinearRegression



import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


from scikeras.wrappers import KerasClassifier, KerasRegressor



from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import sklearn.metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import export_text

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import collections

import time

import pickle





In [ ]:

dscc=pd.read_csv('drive/MyDrive/data/dscc_sub_20240220.csv')
rwd=pd.read_csv('drive/MyDrive/data/rwd_sub_pao2_albumin_20240403.csv')


dscc['pao2']=np.log(dscc['pao2']+1)
rwd['pao2']=np.log(rwd['pao2']+1)

rwd["w"]=0
rwd["y"]=0

dscc["y"]=1-dscc["y"]



In [ ]:

import sys

class output:
    def __init__(self, A_pred, is_vul, obj_all, val_all, obj_vul, val_vul, model_rise):
        self.A_pred = A_pred
        self.is_vul = is_vul
        self.obj_all = obj_all
        self.val_all = val_all
        self.obj_vul = obj_vul
        self.val_vul = val_vul
        self.model = model_rise


class out_model:
    def __init__(self, seed, df, df_test, X_names, y_qua, w_qua, s_type, is_tune, param_grid, transfer_weights):
        self.seed = seed
        self.df = df
        self.df_test = df_test
        self.X_names = X_names
        self.y_qua = y_qua
        self.w_qua = w_qua
        self.s_type = s_type
        self.is_tune = is_tune
        self.param_grid = param_grid

        self.transfer_weights = transfer_weights


In [ ]:

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression


n_MI=5

In [ ]:

import cvxpy as cp
import numpy as np

def entr(w):

    return cp.sum(cp.entr(w))

def entr_balance(tol, x_in_rct, x_in_rwe):
    # the optimization problem for the nonparametric weighting method
    n = x_in_rct.shape[0]
    w = cp.Variable(n)
    p = x_in_rct.shape[1]-1
    objective = cp.Maximize(cp.sum(cp.entr(w)))
    constraints = [w >= 0, cp.sum(w) == 1]
    for k in range(p):
        constraints += [cp.abs(cp.sum(w*x_in_rct[:,k+1]) - np.mean(x_in_rwe[:,k+1])) <= tol*np.std(x_in_rwe[:,k+1])*np.sqrt(n/(n-1))]
        constraints += [cp.abs(cp.sum(w*(x_in_rct[:,k+1]**2)) - np.mean(x_in_rwe[:,k+1]**2)) <= tol*np.std(x_in_rwe[:,k+1]**2)*np.sqrt(n/(n-1))]

    prob = cp.Problem(objective, constraints)
    w_tilde = np.zeros(n)
    try:
        result = prob.solve(solver=cp.ECOS)
        w_tilde = w.value
        if w_tilde is None:
            w_tilde = np.zeros(n)
        else:
            if np.isnan(w_tilde[0]):
                w_tilde = np.zeros(n)

    except cp.error.SolverError as e:

        pass
    return w_tilde

def entr_balance_wmid(tol, x_in_rct, x_in_rwe,wmid):
    # the optimization problem for the nonparametric weighting method
    n = x_in_rct.shape[0]
    wmid0 =  x_in_rwe.shape[0] * wmid
    w = cp.Variable(n)
    p = x_in_rct.shape[1]-1
    objective = cp.Maximize(cp.sum(cp.entr(w)))
    constraints = [w >= 0, cp.sum(w) == 1]
    for k in range(p):
        constraints += [cp.abs(cp.sum(w*x_in_rct[:,k+1]) - np.mean(wmid0*x_in_rwe[:,k+1])) <= tol*np.std(wmid0*x_in_rwe[:,k+1])*np.sqrt(n/(n-1))]
        constraints += [cp.abs(cp.sum(w*(x_in_rct[:,k+1]**2)) - np.mean(wmid0*(x_in_rwe[:,k+1]**2))) <= tol*np.std(wmid0*(x_in_rwe[:,k+1]**2))*np.sqrt(n/(n-1))]

    prob = cp.Problem(objective, constraints)
    w_tilde = np.zeros(n)
    try:
        result = prob.solve(solver=cp.ECOS)
        w_tilde = w.value
        if w_tilde is None:
            w_tilde = np.zeros(n)
        else:
            if np.isnan(w_tilde[0]):
                w_tilde = np.zeros(n)

    except cp.error.SolverError as e:

        pass
    return w_tilde


def weight_tuned_entr(tols, x_in_rct, x_in_rwe, prop=0.5, smps=10):
    # Evaluate covariate balance with bootstrap samples to choose tuning parameters:
    # the degree of approximate balance sigma_k
    n = x_in_rct.shape[0]
    sds = np.apply_along_axis(np.std, 0, x_in_rwe)
    means = np.apply_along_axis(np.mean, 0, x_in_rwe)
    sds2 = np.apply_along_axis(np.std, 0, x_in_rwe**2)
    moment2 = np.apply_along_axis(np.mean, 0, x_in_rwe**2)
    cov_diff_bars = []
    w_tildes = np.full((n, len(tols)), np.nan)
    for i in range(len(tols)):
        tol = tols[i]
        w_tilde = entr_balance(tol, x_in_rct, x_in_rwe)
        w_tilde[w_tilde < 0] = 0  # the optimization results sometimes return negative weights, so clip them to 0
        if np.sum(w_tilde) == 0:
            cov_diff_bar = 1e+10
        else:
            w_tildes[:, i] = w_tilde
            cov_diffs = []
            for s in range(smps):
                boot_ind = np.random.choice(n, size=int(round(prop * n)), replace=True)
                x_in_rct_boot = x_in_rct[boot_ind, :]
                w_tilde_boot = w_tilde[boot_ind]
                cov_diff = (np.sum(x_in_rct_boot * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - means)[1:] / sds[1:]
                cov_diff_moment2 = (np.sum(x_in_rct_boot**2 * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - moment2)[1:] / sds2[1:]

                cov_diffs.append(np.sum(np.concatenate((cov_diff, cov_diff_moment2))**2))  # L2 measure
            cov_diff_bar = np.mean(cov_diffs)
        cov_diff_bars.append(cov_diff_bar)
    tuned_tol = tols[np.argmin(cov_diff_bars)]
    if min(cov_diff_bars) < 1e+10:
        return w_tildes[:, np.argmin(cov_diff_bars)]
    else:
        raise ValueError('************ optimization not feasible *****************')

def weight_tuned_entr_wmid(tols, x_in_rct, x_in_rwe, wmid, prop=0.5, smps=10):
    # Evaluate covariate balance with bootstrap samples to choose tuning parameters:
    # the degree of approximate balance sigma_k
    n = x_in_rct.shape[0]
    wmid0 =  x_in_rwe.shape[0] * wmid
    sds = np.apply_along_axis(np.std, 0, wmid0[:,np.newaxis]*x_in_rwe)

    means = np.apply_along_axis(np.mean, 0, wmid0[:,np.newaxis]*x_in_rwe)
    sds2 = np.apply_along_axis(np.std, 0, wmid0[:,np.newaxis]*(x_in_rwe**2))

    moment2 = np.apply_along_axis(np.mean, 0, wmid0[:,np.newaxis]*(x_in_rwe**2))
    cov_diff_bars = []
    w_tildes = np.full((n, len(tols)), np.nan)
    for i in range(len(tols)):
        tol = tols[i]
        w_tilde = entr_balance_wmid(tol, x_in_rct, x_in_rwe,wmid)

        w_tilde[w_tilde < 0] = 0  # the optimization results sometimes return negative weights, so clip them to 0
        if np.sum(w_tilde) == 0:
            cov_diff_bar = 1e+10
        else:
            w_tildes[:, i] = w_tilde
            cov_diffs = []
            for s in range(smps):

                boot_ind = np.random.choice(n, size=int(round(prop * n)), replace=True)
                x_in_rct_boot = x_in_rct[boot_ind, :]
                w_tilde_boot = w_tilde[boot_ind]
                cov_diff = (np.sum(x_in_rct_boot * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - means)[1:] / sds[1:]
                cov_diff_moment2 = (np.sum(x_in_rct_boot**2 * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - moment2)[1:] / sds2[1:]

                cov_diffs.append(np.sum(np.concatenate((cov_diff, cov_diff_moment2))**2))  # L2 measure
            cov_diff_bar = np.mean(cov_diffs)
        cov_diff_bars.append(cov_diff_bar)
    tuned_tol = tols[np.argmin(cov_diff_bars)]
    if min(cov_diff_bars) < 1e+10:
        return w_tildes[:, np.argmin(cov_diff_bars)]
    else:
        raise ValueError('************ optimization not feasible *****************')


def learn_weights(y_in_rct, x_in_rct, a_in_rct, x_in_rwe, w_method, misspecify, N):
    # w_method: 1, 2, 3, 4
    n = x_in_rct.shape[0]
    p = x_in_rct.shape[1] - 1
    if w_method == 4:
        # nonparametric weighting
        w_tilde = weight_tuned_entr([0.000, 0.001, 0.002, 0.005, 0.010, 0.020, 0.050, 0.100, 0.200, 0.500, 1.000], x_in_rct, x_in_rwe)
    else:
        raise ValueError("Wrong input for weighting method")
    return w_tilde
def learn_weights_wmid(y_in_rct, x_in_rct, a_in_rct, x_in_rwe, w_method, misspecify, N,wmid):
    # w_method: 1, 2, 3, 4
    n = x_in_rct.shape[0]
    p = x_in_rct.shape[1] - 1
    if w_method == 4:
        # nonparametric weighting
        w_tilde = weight_tuned_entr_wmid([0.000, 0.001, 0.002, 0.005, 0.010, 0.020, 0.050, 0.100, 0.200, 0.500, 1.000], x_in_rct, x_in_rwe,wmid)
    else:
        raise ValueError("Wrong input for weighting method")
    return w_tilde



In [ ]:

X_names=["age", "rr","temp","hr","lactate", "hgb","plt","map", "sbp", "male","bili","bun", "creat","gluc","o2_sat","wbc","sofa_total","gcs", "mechvent"]
S_names=["pao2","albumin"]
is_rct=False
is_class=True
s_type="cont"
tw_type=None
qua_use=0.2
is_tune=False
param_grid=None
is_plot=False
method=None
seed=12345

train_df=dscc
test_df=rwd
Y_name="y"
A_name="w"

if (is_tune):

    param_grid = dict(layers=[1],
                      nodes=[256],
                      dropouts=[0.1],
                      acts=["relu"],
                      opts=["adam"],
                      bsizes=[32],
                      n_epochs=[100]
                    )

else:
    param_grid = None

# sanity check
try:
    assert s_type in ["cont", "disc", "multi-disc"]
except:
    sys.exit('s_type should be in ["cont", "disc", "multi-disc"]')

if s_type in ["disc", "multi-disc"]:
    qua_use = None

if is_tune:
    try:
        assert param_grid is not None
    except:
        sys.exit('when is_tune=True, param_grid should not be None')
else:
    param_grid = None

# default values
is_sim = False
proj, y_fn = None, None

train_df=train_df.copy()
test_df=test_df.copy()
# reorganize data
train_df.rename(columns={Y_name:"Y"}, inplace=True)
test_df.rename(columns={Y_name:"Y"}, inplace=True)

train_df.rename(columns={A_name:"A"}, inplace=True)
test_df.rename(columns={A_name:"A"}, inplace=True)
if set(train_df["A"]) != set([0,1]): # randomly pick one as control/treatment
    assert set(train_df["A"]) == set(test_df["A"])
    a0 = list(set(train_df["A"]))[0]
    train_df["A"] = np.where(train_df["A"]==a0, 0, 1)
    test_df["A"] = np.where(test_df["A"]==a0, 0, 1)

XS_names = S_names + X_names

# discrete var has <=5 values #TODO: may need to change in future
if s_type == "cont":
    names_to_norm = list(train_df[XS_names].nunique()[train_df[XS_names].nunique() > 5].index)
else: # not normalize S if disc/multi-disc
    names_to_norm = list(train_df[X_names].nunique()[train_df[X_names].nunique() > 5].index)

set_random(seed)


############################################################################
################################ Preprocess ################################
############################################################################

# normalize
df, df_test = normalize(train_df, test_df, names_to_norm)


df_RCT=df.copy()




df_RWD_tosplit=df_test.copy()

df_RWD_wtcalc = df_RWD_tosplit[df_RWD_tosplit["two_status"]==1]
df_RWD = df_RWD_tosplit[df_RWD_tosplit["two_status"]==0]

df_RWD_forweights=pd.concat([df_RWD_wtcalc.copy(),df_RCT.copy()]).reset_index(drop=True).copy()


print("df_RWD_forweights number of rows",df_RWD_forweights.shape[0])

df_pop=df_RWD.copy()


def transfer_weight_obj(alpha):
    m=df_RWD_forweights.shape[0]
    n=df_RCT.shape[0]

    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
    df_RWD_1X=np.concatenate((np.ones((df_RWD_forweights.shape[0],1)),df_RWD_forweights[XS_names]),axis=1)

    return -(1/m+n)*np.sum(df_RCT_1X@alpha)+(1/m+n)*np.sum(np.log(1+np.exp(df_RWD_1X@alpha)))




alpha_sol=scipy.optimize.minimize(transfer_weight_obj, x0=np.asarray([0 for i in range(np.asarray(XS_names).shape[0]+1)]), method='Nelder-Mead', tol=1e-6)
alpha_sol=alpha_sol.x



df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)

transfer_weights=np.exp(-df_RCT_1X@alpha_sol)




alpha_true=np.array([-2,-4])

df_RCT["TW_test"]=transfer_weights.copy()
df_RCT["TW"]=transfer_weights
df_RCT["TW_ones"]=np.ones(df_RCT.shape[0])


is_ones="impute"

df_RWD_tosplit["pMCAR"] = (df_RWD_tosplit["two_status"]==0)
if is_ones=="complete":
    wt_type="complete"
    df_RWD_wtcalc = df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0]
if is_ones=="impute" or is_ones=="np_impute_old":
    wt_type="impute"
    df_RWD_wtcalc = df_RWD_tosplit.copy()
if is_ones=="complete" or is_ones=="impute":
    df_RWD_forweights=pd.concat([df_RWD_wtcalc.copy(),df_RCT.copy()]).reset_index(drop=True).copy()
    print("df_RWD_forweights number of rows",df_RWD_forweights.shape[0])


    def transfer_weight_obj(alpha):
        m=df_RWD_forweights.shape[0]
        n=df_RCT.shape[0]



        df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
        df_RWD_1X=np.concatenate((np.ones((df_RWD_forweights.shape[0],1)),df_RWD_forweights[XS_names]),axis=1)

        return -(1/m+n)*np.sum(df_RCT_1X@alpha)+(1/m+n)*np.sum(np.log(1+np.exp(df_RWD_1X@alpha)))



    alpha_sol=scipy.optimize.minimize(transfer_weight_obj, x0=np.asarray([0 for i in range(np.asarray(XS_names).shape[0]+1)]), method='Nelder-Mead', tol=1e-6)
    alpha_sol=alpha_sol.x




    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)

    transfer_weights=np.exp(-df_RCT_1X@alpha_sol)

    alpha_true=np.array([-2,-4])

    print(df_RCT.shape[0],df_RWD.shape[0])
    print(alpha_true)
    print(np.around(alpha_sol,4))

if is_ones=="impute_only_no":
    wt_type="impute_only"
    df_RWD_tosplit_impute = df_RWD_tosplit.copy()
    df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1]=np.nan

    rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_classifier.fit(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==0], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==0])
    df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1] = rf_classifier.predict(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==1])

    impute_accuracy = mean_squared_error(df_RWD_tosplit["S"][df_RWD_tosplit["pMCAR"]==1], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1])

    print(f'ImputeAccuracy: {impute_accuracy}')
    print("*****")

    df_RWD_wtcalc = df_RWD_tosplit_impute[df_RWD_tosplit["pMCAR"]==1].copy()

if is_ones=="impute_only":
    wt_type="impute_only"
    df_RWD_impar = df_RWD_tosplit[XS_names].copy()  # Select the XS_names columns from RWD
    df_RWD_impar['Source'] = 0  # Add 'Source' column indicating RWD data
    df_RCT_impar = df_RCT[XS_names].copy()  # Select the XS_names columns from RCT
    df_RCT_impar['Source'] = 1  # Add 'Source' column indicating RCT data
    df_impar = pd.concat([df_RWD_impar, df_RCT_impar], axis=0).reset_index(drop=True)
    rf_classifier1 = RandomForestClassifier(random_state=42)
    rf_classifier1.fit(df_impar[X_names], df_impar['Source'])
    prob_RCT1 = np.clip(rf_classifier1.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)
    log_regressor1 = LogisticRegression(random_state=42)
    log_regressor1.fit(df_impar[X_names], df_impar['Source'])
    prob_RCT1 = np.clip(log_regressor1.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)

    df_RWD_impar_com = df_RWD_tosplit.loc[df_RWD_tosplit["pMCAR"] == 1, XS_names].copy()
    df_RWD_impar_com['Source'] = 0  # Add 'Source' column indicating RWD data
    df_impar_com = pd.concat([df_RWD_impar_com, df_RCT_impar], axis=0).reset_index(drop=True)
    rf_classifier2 = RandomForestClassifier(random_state=42)
    rf_classifier2.fit(df_impar_com[X_names], df_impar_com['Source'])
    prob_RCT2 = np.clip(rf_classifier2.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)
    log_regressor2 = LogisticRegression(random_state=42)
    log_regressor2.fit(df_impar_com[X_names], df_impar_com['Source'])
    prob_RCT2 = np.clip(log_regressor2.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)

    rf_classifier3 = RandomForestClassifier(random_state=42)
    rf_classifier3.fit(df_impar_com[XS_names], df_impar_com['Source'])
    prob_RCT3 = np.clip(rf_classifier3.predict_proba(df_RCT[XS_names]), 1e-5, 1-1e-5)
    log_regressor3 = LogisticRegression(random_state=42)
    log_regressor3.fit(df_impar_com[XS_names], df_impar_com['Source'])
    prob_RCT3 = np.clip(log_regressor3.predict_proba(df_RCT[XS_names]), 1e-5, 1-1e-5)



    transfer_weights = (prob_RCT1[:, 0]*prob_RCT2[:, 1]*prob_RCT3[:, 0])/(prob_RCT1[:, 1]*prob_RCT2[:, 0]*prob_RCT3[:, 1])

if is_ones=="np":
    df_RWD_wtcalc = df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0]
    df_RWD_forweights_2=df_RWD_wtcalc.copy().reset_index(drop=True)
    df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
    transfer_weights=learn_weights(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0)
    wt_type="np"
if is_ones=="np_impute_old":
    wt_type="np_impute_old"
    df_RWD_forweights_2=df_RWD_wtcalc.copy().reset_index(drop=True)
    df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
    transfer_weights=learn_weights(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0)
if is_ones=="np_impute":

    df_RWD_forweights_2=df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0].copy().reset_index(drop=True)
    df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
    df_RWD_1X_2_Xonly=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[X_names]),axis=1)
    df_RWD_forweights_2_all=df_RWD_tosplit.copy().reset_index(drop=True)
    df_RWD_1X_2_all=np.concatenate((np.ones((df_RWD_forweights_2_all.shape[0],1)),df_RWD_forweights_2_all[X_names]),axis=1)
    transfer_weights_wmid=learn_weights(y_in_rct=0, x_in_rct=df_RWD_1X_2_Xonly, a_in_rct=0, x_in_rwe=df_RWD_1X_2_all, w_method=4, misspecify=0, N=0)
    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
    transfer_weights=learn_weights_wmid(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0,wmid=transfer_weights_wmid)
    wt_type="np_impute"



df_RCT["TW_test"]=transfer_weights.copy()
df_RCT["TW"]=transfer_weights
df_RCT["TW_ones"]=np.ones(df_RCT.shape[0])

df_tr_ori = df_RCT.reset_index(drop=True).copy()
df_te_ori = df_RWD.reset_index(drop=True).copy()


df=df_RCT.reset_index(drop=True).copy()
df_test=df_RWD.reset_index(drop=True).copy()

df_pop=df_RWD.reset_index(drop=True).copy()


# PS based on train
if is_rct:
    pr_a1 = sum(df["A"]==1) / df.shape[0]
    fit_ps = None
    df_test["ps"] = pr_a1
    df["ps"] = pr_a1
    print("pr_a1", pr_a1)
else:
    pr_a1 = None
    fit_ps = get_ps_fit(df, XS_names, transfer_weights=df["TW"]) # propensity score A|XS
    df_test["ps"] = fit_ps.predict_proba(df_test[XS_names])[:,1]
    df["ps"] = fit_ps.predict_proba(df[XS_names])[:,1]

df_test["ps"] = np.where(df_test["A"]==1, df_test["ps"], 1-df_test["ps"])
df["ps"] = np.where(df["A"]==1, df["ps"], 1-df["ps"])


if not is_rct:
    print("for observational study, trimming PS using percentile cutpoints")

    ps_q90 = df["ps"].quantile(0.9)
    ps_q10 = df["ps"].quantile(0.1)
    print("train: before ps min:", np.min(df["ps"]), "ps max:", np.max(df["ps"]))
    df["ps"] = np.where(df["ps"] > ps_q90, ps_q90, df["ps"])
    df["ps"] = np.where(df["ps"] < ps_q10, ps_q10, df["ps"])
    print("train: after  ps min:", np.min(df["ps"]), "ps max:", np.max(df["ps"]))

    ps_q90 = df_test["ps"].quantile(0.9)
    ps_q10 = df_test["ps"].quantile(0.1)
    print("test: before ps min:", np.min(df_test["ps"]), "ps max:", np.max(df_test["ps"]))
    df_test["ps"] = np.where(df_test["ps"] > ps_q90, ps_q90, df_test["ps"])
    df_test["ps"] = np.where(df_test["ps"] < ps_q10, ps_q10, df_test["ps"])
    print("test: after  ps min:", np.min(df_test["ps"]), "ps max:", np.max(df_test["ps"]))


############################################################################
############################## Begin Analysis ##############################
############################################################################

df0 = df[df["A"]==0].reset_index(drop=True).copy()
df1 = df[df["A"]==1].reset_index(drop=True).copy()

df0_te = df_test[df_test["A"]==0].reset_index(drop=True).copy()
df1_te = df_test[df_test["A"]==1].reset_index(drop=True).copy()


if df1_te.shape[0]<1:
    df1_te=None

df1_te=None
df0_te=None



obj_name = "_expOracle_"


g_exp0, fit_Y_XS_A0 = fit_expectation(obj_name, df0, XS_names, df, is_class, is_tune, param_grid, df0["TW"], df0_te)
g_exp1, fit_Y_XS_A1 = fit_expectation(obj_name, df1, XS_names, df, is_class, is_tune, param_grid, df1["TW"], df1_te)

g_exp0_ones, fit_Y_XS_A0_ones = fit_expectation(obj_name, df0, XS_names, df, is_class, is_tune, param_grid, df0["TW_ones"], df0_te)
g_exp1_ones, fit_Y_XS_A1_ones = fit_expectation(obj_name, df1, XS_names, df, is_class, is_tune, param_grid, df1["TW_ones"], df1_te)


y_exp, w_exp = get_label_wt(g_exp1, g_exp0)
A_te_expOracle, A_tr_expOracle, model_rise_expOracle = class_pred(df, df_test, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])

y_exp_ones, w_exp_ones = get_label_wt(g_exp1_ones, g_exp0_ones)
A_te_expOracle_ones, A_tr_expOracle_ones, model_rise_expOracle_ones = class_pred(df, df_test, XS_names, y_exp_ones, w_exp_ones, s_type, is_tune, param_grid, df["TW_ones"])

all_A_expOracle_MI = []  # list to store each A_expOracle_MI
all_A_expOracle_MI_ones = []

model_expOracle_MI = []  # list to store each A_expOracle_MI
model_expOracle_MI_ones = []
# Example loop (replace with your actual loop where A_expOracle_MI is generated)
for reprep in range(n_MI):  # num_loops is the number of iterations

    df_RWD_tosplit_impute_MI = df_RWD_tosplit.copy()
    df_RWD_tosplit_impute_MI[S_names][df_RWD_tosplit["two_status"]==0]=np.nan


    rf_classifierS1 = RandomForestRegressor(n_estimators=100, random_state=42+reprep*100)
    rf_classifierS1.fit(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["two_status"]==1], df_RWD_tosplit_impute_MI[S_names[0]][df_RWD_tosplit["two_status"]==1])
    df_RWD_tosplit_impute_MI[S_names[0]][df_RWD_tosplit["two_status"]==0] = rf_classifierS1.predict(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["two_status"]==0])
    rf_classifierS2 = RandomForestRegressor(n_estimators=100, random_state=42+reprep*100)
    rf_classifierS2.fit(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["two_status"]==1], df_RWD_tosplit_impute_MI[S_names[1]][df_RWD_tosplit["two_status"]==1])
    df_RWD_tosplit_impute_MI[S_names[1]][df_RWD_tosplit["two_status"]==0] = rf_classifierS2.predict(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["two_status"]==0])
    df_RWD_MI = df_RWD_tosplit_impute_MI.copy()
    df_RWD_MI = df_RWD_MI[df_RWD_tosplit["two_status"]==0]
    df_test_MI = df_RWD_MI.reset_index(drop=True).copy()

    A_expOracle_MI, _, model_expOracle_MI_0 = class_pred(df, df_test_MI, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])
    A_expOracle_MI_ones, _, model_expOracle_MI_ones_0 = class_pred(df, df_test_MI, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW_ones"])
    all_A_expOracle_MI_ones.append(A_expOracle_MI_ones)
    all_A_expOracle_MI.append(A_expOracle_MI)
    model_expOracle_MI.append(model_expOracle_MI_0)
    model_expOracle_MI_ones.append(model_expOracle_MI_ones_0)

# Convert list of arrays into a 2D NumPy array
all_A_expOracle_MI = np.array(all_A_expOracle_MI)  # Shape: (num_loops, array_length)
all_A_expOracle_MI_ones = np.array(all_A_expOracle_MI_ones)

# Now, find the majority vote for each position
majority_vote = []
majority_vote_ones = []
for col in range(all_A_expOracle_MI.shape[1]):  # Iterate over each column (position)
    votes = all_A_expOracle_MI[:, col]  # Get the votes for the current position
    most_common_vote = Counter(votes).most_common(1)[0][0]  # Get the majority vote
    majority_vote.append(most_common_vote)

for col in range(all_A_expOracle_MI_ones.shape[1]):  # Iterate over each column (position)
    votes_ones = all_A_expOracle_MI_ones[:, col]  # Get the votes for the current position
    most_common_vote_ones = Counter(votes_ones).most_common(1)[0][0]  # Get the majority vote
    majority_vote_ones.append(most_common_vote_ones)

# Convert the result to a NumPy array (if needed)
final_majority_vote = np.array(majority_vote)
A_MI = final_majority_vote
final_majority_vote_ones = np.array(majority_vote_ones)
A_MI_ones = final_majority_vote_ones


# QUA/INF E(Y|X,S,A)
set_random(seed)
df0["Yhat"] = fit_Y_XS_A0.predict_proba(df0[XS_names])[:,1]
df1["Yhat"] = fit_Y_XS_A1.predict_proba(df1[XS_names])[:,1]
df0["Yhat_ones"] = fit_Y_XS_A0_ones.predict_proba(df0[XS_names])[:,1]
df1["Yhat_ones"] = fit_Y_XS_A1_ones.predict_proba(df1[XS_names])[:,1]

fit_qua0, fit_qua1 = None, None

fit_qua0_test, fit_qua1_test = None, None
fit_qua0_ones, fit_qua1_ones = None, None
if (s_type == "cont"): # quantile: Yhat|X,A (no S)
    set_random(seed)
    fit_qua0, g_qua0 = fit_quantile(df0[X_names], df0["Yhat"], df[X_names], qua_use, is_tune, param_grid, df0["TW"])
    fit_qua1, g_qua1 = fit_quantile(df1[X_names], df1["Yhat"], df[X_names], qua_use, is_tune, param_grid, df1["TW"])

    fit_qua0_ones, g_qua0_ones = fit_quantile(df0[X_names], df0["Yhat_ones"], df[X_names], qua_use, is_tune, param_grid, df0["TW_ones"])
    fit_qua1_ones, g_qua1_ones = fit_quantile(df1[X_names], df1["Yhat_ones"], df[X_names], qua_use, is_tune, param_grid, df1["TW_ones"])

if (s_type == "disc"): # infinite: try all s and find s that helps achieve a minimal
    g_qua0 = fit_infinite(df, X_names, XS_names, fit_Y_XS_A0)
    g_qua1 = fit_infinite(df, X_names, XS_names, fit_Y_XS_A1)

y_qua, w_qua = get_label_wt(g_qua1, g_qua0)
y_qua_ones, w_qua_ones = get_label_wt(g_qua1_ones, g_qua0_ones)
set_random(seed)



A_te, A_tr, model_rise = class_pred(df, df_test, X_names, y_qua, w_qua, s_type, is_tune, param_grid, df["TW"],S_names)
A_te_ones, A_tr_ones, model_rise_ones = class_pred(df, df_test, X_names, y_qua_ones, w_qua_ones, s_type, is_tune, param_grid, df["TW_ones"],S_names)




# EXP E(Y|X,S,A) # XS train, XS predict

obj_name = "_base_"
g_base0, _ = fit_expectation(obj_name, df0, X_names, df, is_class, is_tune, param_grid, df0["TW"], df0_te)
g_base1, _ = fit_expectation(obj_name, df1, X_names, df, is_class, is_tune, param_grid, df1["TW"], df1_te)

y_base, w_base = get_label_wt(g_base1, g_base0)
A_te_base, A_tr_base, model_rise_base = class_pred(df, df_test, X_names, y_base, w_base, s_type, is_tune, param_grid, df["TW"])

g_base0_ones, _ = fit_expectation(obj_name, df0, X_names, df, is_class, is_tune, param_grid, df0["TW_ones"], df0_te)
g_base1_ones, _ = fit_expectation(obj_name, df1, X_names, df, is_class, is_tune, param_grid, df1["TW_ones"], df1_te)

y_base_ones, w_base_ones = get_label_wt(g_base1_ones, g_base0_ones)
A_te_base_ones, A_tr_base_ones, model_rise_base_ones = class_pred(df, df_test, X_names, y_base_ones, w_base_ones, s_type, is_tune, param_grid, df["TW_ones"])

obj_name = "_exp_"
A_te_exp, A_tr_exp, model_rise_exp = class_pred(df, df_test, X_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])
A_te_exp_ones, A_tr_exp_ones, model_rise_exp_ones = class_pred(df, df_test, X_names, y_exp_ones, w_exp_ones, s_type, is_tune, param_grid, df["TW_ones"])



A_te_obs = np.where(df_test["A"]==0,"yes","no")

if is_plot:

    out_mod = out_model(seed, df, df_test, X_names, y_qua, w_qua, s_type, is_tune, param_grid, df["TW"])
    if (method == "expOracle"):
        out_mod = out_model(seed, df, df_test, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])
    if (method == "base"):
        out_mod = out_model(seed, df, df_test, X_names, y_base, w_base, s_type, is_tune, param_grid, df["TW"])
    if (method == "exp"):
        out_mod = out_model(seed, df, df_test,X_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])

else:
    out_mod = None
out_mod_ones=None



In [ ]:

############################################################################
############################# Evaluate Metrics #############################
############################################################################

pqa=0.2

fit_qua0_pqa, _ = fit_quantile(df0[X_names], df0["Yhat"], df[X_names], pqa, is_tune, param_grid, df0["TW"])
fit_qua1_pqa, _ = fit_quantile(df1[X_names], df1["Yhat"], df[X_names], pqa, is_tune, param_grid, df1["TW"])


A_name = 'A_rise'
M_name = 'M_opt'
df_test[A_name] = A_te

A_te_MI=A_MI
df_test['A_obs'] = A_te_obs
df_test['A_base'] = A_te_base
df_test['A_exp'] = A_te_exp
df_test['A_MI'] = A_MI
df_test['A_expOracle'] = A_te_expOracle

df_test['A_base_ones'] = A_te_base_ones
df_test['A_exp_ones'] = A_te_exp_ones
df_test['A_MI_ones'] = A_MI_ones
df_test['A_expOracle_ones'] = A_te_expOracle_ones
df_test['A_rise_ones'] = A_te_ones



df_test['Yhat_XS_A0'] = fit_Y_XS_A0.predict_proba(df_test[XS_names])[:,1]
df_test['Yhat_XS_A1'] = fit_Y_XS_A1.predict_proba(df_test[XS_names])[:,1]



def invlogit(x):
    return 1 / (1 + np.exp(-x))
def logit(p):
    return np.log(p / (1 - p))
df_test['q_A0'] = invlogit(fit_qua0_pqa.predict(df_test[X_names]))


df_test['q_A1'] = invlogit(fit_qua1_pqa.predict(df_test[X_names]))


if s_type in ["cont", "disc"]:


    df_test[M_name]=optimal_AS_inf(df_test, proj, is_sim, s_type, X_names, XS_names, y_fn,
                   is_class, pqa, fit_Y_XS_A0, fit_Y_XS_A1, fit_qua0, fit_qua1, fit_qua0_pqa, fit_qua1_pqa)
    df_predvul=df_test.copy()
    df_predvul["weights"]=1
    print("20240220*************")

    print(np.mean(df_test[M_name]))

else: # "multi-disc"
    df_test[M_name] = optimal_AS_inf_mulS(df_test, proj, is_sim, s_type, X_names,
                XS_names, y_fn, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)




_, _, fit_vul_XS = class_pred(df_predvul, df_predvul, XS_names, df_predvul[M_name], df_predvul["weights"], s_type, is_tune, param_grid, df_predvul["weights"])

plot_shap(fit_vul_XS,XS_names,df_predvul, title="key factors of vulnerable samples")


_, _, fit_vul_X = class_pred(df_predvul, df_predvul, X_names, df_predvul[M_name], df_predvul["weights"], s_type, is_tune, param_grid, df_predvul["weights"])

plot_shap(fit_vul_X,X_names,df_predvul, title="key factors of vulnerable samples")

# metrics
if is_rct:
    Rval_all, Rval_vul, Rvalue_rest = expect_reward(df_test, A_name, M_name)
    Rval_all_obs, Rval_vul_obs, Rvalue_rest_obs = expect_reward(df_test, 'A_obs', M_name)
    Rval_all_base, Rval_vul_base, Rvalue_rest_base = expect_reward(df_test, 'A_base', M_name)
    Rval_all_exp, Rval_vul_exp, Rvalue_rest_exp = expect_reward(df_test, 'A_exp', M_name)
    Rval_all_expOracle, Rval_vul_expOracle, Rvalue_rest_expOracle = expect_reward(df_test, 'A_expOracle', M_name)

    val_all, val_vul, value_rest, _ = get_value_fn(df_test, A_name, XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_obs, val_vul_obs, value_rest_obs, _ = get_value_fn(df_test, 'A_obs', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_base, val_vul_base, value_rest_base, _ = get_value_fn(df_test, 'A_base', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_exp, val_vul_exp, value_rest_exp, _ = get_value_fn(df_test, 'A_exp', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_MI, val_vul_MI, value_rest_MI, _ = get_value_fn(df_test, 'A_MI', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_expOracle, val_vul_expOracle, value_rest_expOracle, _ = get_value_fn(df_test, 'A_expOracle', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

    Rval_all_ones, Rval_vul_ones, Rvalue_rest_ones = expect_reward(df_test, 'A_rise_ones', M_name)
    Rval_all_base_ones, Rval_vul_base_ones, Rvalue_rest_base_ones = expect_reward(df_test, 'A_base_ones', M_name)
    Rval_all_exp_ones, Rval_vul_exp_ones, Rvalue_rest_exp_ones = expect_reward(df_test, 'A_exp_ones', M_name)
    Rval_all_expOracle_ones, Rval_vul_expOracle_ones, Rvalue_rest_expOracle_ones = expect_reward(df_test, 'A_expOracle_ones', M_name)

    val_all_ones, val_vul_ones, value_rest_ones, _ = get_value_fn(df_test, 'A_rise_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_base_ones, val_vul_base_ones, value_rest_base_ones, _ = get_value_fn(df_test, 'A_base_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_exp_ones, val_vul_exp_ones, value_rest_exp_ones, _ = get_value_fn(df_test, 'A_exp_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_MI_ones, val_vul_MI_ones, value_rest_MI_ones, _ = get_value_fn(df_test, 'A_MI_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_expOracle_ones, val_vul_expOracle_ones, value_rest_expOracle_ones, _ = get_value_fn(df_test, 'A_expOracle_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

else:

    val_all, val_vul, value_rest, _ = get_value_fn(df_test, A_name, XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_obs, val_vul_obs, value_rest_obs, _ = get_value_fn(df_test, 'A_obs', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_base, val_vul_base, value_rest_base, _ = get_value_fn(df_test, 'A_base', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_exp, val_vul_exp, value_rest_exp, _ = get_value_fn(df_test, 'A_exp', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_MI, val_vul_MI, value_rest_MI, _ = get_value_fn(df_test, 'A_MI', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_expOracle, val_vul_expOracle, value_rest_expOracle, _ = get_value_fn(df_test, 'A_expOracle', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

    val_all_ones, val_vul_ones, value_rest_ones, _ = get_value_fn(df_test, 'A_rise_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_base_ones, val_vul_base_ones, value_rest_base_ones, _ = get_value_fn(df_test, 'A_base_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_exp_ones, val_vul_exp_ones, value_rest_exp_ones, _ = get_value_fn(df_test, 'A_exp_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_MI_ones, val_vul_MI_ones, value_rest_MI_ones, _ = get_value_fn(df_test, 'A_MI_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)
    val_all_expOracle_ones, val_vul_expOracle_ones, value_rest_expOracle_ones, _ = get_value_fn(df_test, 'A_expOracle_ones', XS_names, M_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

if s_type == "disc" or y_fn is not None: # real with disc S | sim with disc/cont S
    obj_all, obj_vul, obj_rest = get_obj_inf(df_test, X_names, XS_names, A_name, M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_obs, obj_vul_obs, obj_rest_obs = get_obj_inf(df_test, X_names, XS_names, 'A_obs', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_base, obj_vul_base, obj_rest_base = get_obj_inf(df_test, X_names, XS_names, 'A_base', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_exp, obj_vul_exp, obj_rest_exp = get_obj_inf(df_test, X_names, XS_names, 'A_exp', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_expOracle, obj_vul_expOracle, obj_rest_expOracle = get_obj_inf(df_test, X_names, XS_names, 'A_expOracle', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)

    obj_all_ones, obj_vul_ones, obj_rest_ones = get_obj_inf(df_test, X_names, XS_names, 'A_rise_ones', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_base_ones, obj_vul_base_ones, obj_rest_base_ones = get_obj_inf(df_test, X_names, XS_names, 'A_base_ones', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_exp_ones, obj_vul_exp_ones, obj_rest_exp_ones = get_obj_inf(df_test, X_names, XS_names, 'A_exp_ones', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
    obj_all_expOracle_ones, obj_vul_expOracle_ones, obj_rest_expOracle_ones = get_obj_inf(df_test, X_names, XS_names, 'A_expOracle_ones', M_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)

elif s_type == "cont": # real with cont S
    obj_all, obj_vul, obj_rest = get_obj_qua(df_test, X_names, s_type, A_name, M_name, fit_qua0, fit_qua1)
    obj_all_obs, obj_vul_obs, obj_rest_obs = get_obj_qua(df_test, X_names, s_type, 'A_obs', M_name, fit_qua0, fit_qua1)
    obj_all_base, obj_vul_base, obj_rest_base = get_obj_qua(df_test, X_names, s_type, 'A_base', M_name, fit_qua0, fit_qua1)
    obj_all_exp, obj_vul_exp, obj_rest_exp = get_obj_qua(df_test, X_names, s_type, 'A_exp', M_name, fit_qua0, fit_qua1)
    obj_all_MI, obj_vul_MI, obj_rest_MI = get_obj_qua(df_test, X_names, s_type, 'A_MI', M_name, fit_qua0, fit_qua1)
    obj_all_expOracle, obj_vul_expOracle, obj_rest_expOracle = get_obj_qua(df_test, X_names, s_type, 'A_expOracle', M_name, fit_qua0, fit_qua1)

    obj_all_ones, obj_vul_ones, obj_rest_ones = get_obj_qua(df_test, X_names, s_type, 'A_rise_ones', M_name, fit_qua0, fit_qua1)
    obj_all_base_ones, obj_vul_base_ones, obj_rest_base_ones = get_obj_qua(df_test, X_names, s_type, 'A_base_ones', M_name, fit_qua0, fit_qua1)
    obj_all_exp_ones, obj_vul_exp_ones, obj_rest_exp_ones = get_obj_qua(df_test, X_names, s_type, 'A_exp_ones', M_name, fit_qua0, fit_qua1)
    obj_all_MI_ones, obj_vul_MI_ones, obj_rest_MI_ones = get_obj_qua(df_test, X_names, s_type, 'A_MI_ones', M_name, fit_qua0, fit_qua1)
    obj_all_expOracle_ones, obj_vul_expOracle_ones, obj_rest_expOracle_ones = get_obj_qua(df_test, X_names, s_type, 'A_expOracle_ones', M_name, fit_qua0, fit_qua1)

# out
A_pred = df_test[A_name].values
A_pred_obs = df_test['A_obs'].values
A_pred_base = df_test['A_base'].values
A_pred_exp = df_test['A_exp'].values
A_pred_expOracle = df_test['A_expOracle'].values

A_pred_ones = df_test['A_rise_ones'].values
A_pred_base_ones = df_test['A_base_ones'].values
A_pred_exp_ones = df_test['A_exp_ones'].values
A_pred_expOracle_ones = df_test['A_expOracle_ones'].values

is_vul = df_test[M_name].values
out_res = output(A_pred, is_vul, obj_all, val_all, obj_vul, val_vul, model_rise)
out_res_obs = output(A_pred_obs, is_vul, obj_all_obs, val_all_obs, obj_vul_obs, val_vul_obs, model_rise)
out_res_base = output(A_pred_base, is_vul, obj_all_base, val_all_base, obj_vul_base, val_vul_base, model_rise_base)
out_res_exp = output(A_pred_exp, is_vul, obj_all_exp, val_all_exp, obj_vul_exp, val_vul_exp, model_rise_exp)
out_res_MI = output(A_pred_exp, is_vul, obj_all_MI, val_all_MI, obj_vul_MI, val_vul_MI, model_rise_exp)
out_res_expOracle = output(A_pred_expOracle, is_vul, obj_all_expOracle, val_all_expOracle, obj_vul_expOracle, val_vul_expOracle, model_rise_expOracle)

out_res_ones = output(A_pred_ones, is_vul, obj_all_ones, val_all_ones, obj_vul_ones, val_vul_ones, model_rise_ones)
out_res_base_ones = output(A_pred_base_ones, is_vul, obj_all_base_ones, val_all_base_ones, obj_vul_base_ones, val_vul_base_ones, model_rise_base_ones)
out_res_exp_ones = output(A_pred_exp_ones, is_vul, obj_all_exp_ones, val_all_exp_ones, obj_vul_exp_ones, val_vul_exp_ones, model_rise_exp_ones)
out_res_MI_ones = output(A_pred_exp_ones, is_vul, obj_all_MI_ones, val_all_MI_ones, obj_vul_MI_ones, val_vul_MI_ones, model_rise_exp_ones)
out_res_expOracle_ones = output(A_pred_expOracle_ones, is_vul, obj_all_expOracle_ones, val_all_expOracle_ones, obj_vul_expOracle_ones, val_vul_expOracle_ones, model_rise_expOracle_ones)

if is_rct:
    Rout_res = output(A_pred, is_vul, obj_all, Rval_all, obj_vul, Rval_vul, model_rise)
    Rout_res_obs = output(A_pred_obs, is_vul, obj_all_obs, Rval_all_obs, obj_vul_obs, Rval_vul_obs, model_rise)
    Rout_res_base = output(A_pred_base, is_vul, obj_all_base, Rval_all_base, obj_vul_base, Rval_vul_base, model_rise_base)
    Rout_res_exp = output(A_pred_exp, is_vul, obj_all_exp, Rval_all_exp, obj_vul_exp, Rval_vul_exp, model_rise_exp)
    Rout_res_MI = output(A_pred_exp, is_vul, obj_all_exp, Rval_all_MI, obj_vul_exp, Rval_vul_MI, model_rise_exp)
    Rout_res_expOracle = output(A_pred_expOracle, is_vul, obj_all_expOracle, Rval_all_expOracle, obj_vul_expOracle, Rval_vul_expOracle, model_rise_expOracle)

    Rout_res_ones = output(A_pred_ones, is_vul, obj_all_ones, Rval_all_ones, obj_vul_ones, Rval_vul_ones, model_rise_ones)
    Rout_res_base_ones = output(A_pred_base_ones, is_vul, obj_all_base_ones, Rval_all_base_ones, obj_vul_base_ones, Rval_vul_base_ones, model_rise_base_ones)
    Rout_res_exp_ones = output(A_pred_exp_ones, is_vul, obj_all_exp_ones, Rval_all_exp_ones, obj_vul_exp_ones, Rval_vul_exp_ones, model_rise_exp_ones)
    Rout_res_expOracle_ones = output(A_pred_expOracle_ones, is_vul, obj_all_expOracle_ones, Rval_all_expOracle_ones, obj_vul_expOracle_ones, Rval_vul_expOracle_ones, model_rise_expOracle_ones)







plot_shap(model_rise_exp_ones,X_names,df_test, title="mean")
plot_shap(model_rise_exp,X_names,df_test, title="mean-tl")

plot_shap(model_rise_expOracle_ones,XS_names,df_test, title="full")
plot_shap(model_rise_expOracle,XS_names,df_test, title="full-tl")

plot_shap(model_rise_ones,X_names,df_test, title="robust")
plot_shap(model_rise,X_names,df_test, title="robust-tl")

if is_rct:
    out_res_rise, out_mod, out_res_obs, out_res_base, out_res_exp, out_res_expOracle, out_res_rise_ones, out_mod_ones, out_res_base_ones, out_res_exp_ones, out_res_expOracle_ones = out_res, out_mod, out_res_obs, out_res_base, out_res_exp, out_res_expOracle, Rout_res, Rout_res_obs, Rout_res_base, Rout_res_exp, Rout_res_expOracle, out_res_ones, out_mod_ones, out_res_base_ones, out_res_exp_ones, out_res_expOracle_ones, Rout_res_ones, Rout_res_base_ones, Rout_res_exp_ones, Rout_res_expOracle_ones
else:
    out_res_rise, out_mod, out_res_obs, out_res_base, out_res_exp, out_res_expOracle, out_res_rise_ones, out_mod_ones, out_res_base_ones, out_res_exp_ones, out_res_expOracle_ones = out_res, out_mod, out_res_obs, out_res_base, out_res_exp, out_res_expOracle, out_res_ones, out_mod_ones, out_res_base_ones, out_res_exp_ones, out_res_expOracle_ones

print("obs")
print("obj_all:", np.round(out_res_obs.obj_all,2))
print("obj_vul:", np.round(out_res_obs.obj_vul,2))
print("val_all:", np.round(out_res_obs.val_all,2))
print("val_vul:", np.round(out_res_obs.val_vul,2))
print("num_trt:", out_res_rise.A_pred[np.where(out_res_obs.A_pred=="yes")].shape[0])


print("base_ones")
print("obj_all:", np.round(out_res_base_ones.obj_all,2))
print("obj_vul:", np.round(out_res_base_ones.obj_vul,2))
print("val_all:", np.round(out_res_base_ones.val_all,2))
print("val_vul:", np.round(out_res_base_ones.val_vul,2))
print("num_vul:", out_res_base.is_vul[np.where(out_res_base.is_vul==1)].shape[0])
print("num_trt:", out_res_base.A_pred[np.where(out_res_base_ones.A_pred=="yes")].shape[0])


print("exp_ones")
print("obj_all:", np.round(out_res_exp_ones.obj_all,3))
print("obj_vul:", np.round(out_res_exp_ones.obj_vul,3))
print("val_all:", np.round(out_res_exp_ones.val_all,3))
print("val_vul:", np.round(out_res_exp_ones.val_vul,3))
print("num_vul:", out_res_exp.is_vul[np.where(out_res_exp.is_vul==1)].shape[0])
print("num_trt:", out_res_exp.A_pred[np.where(out_res_exp_ones.A_pred=="yes")].shape[0])

print("MI_ones")
print("obj_all:", np.round(out_res_MI_ones.obj_all,3))
print("obj_vul:", np.round(out_res_MI_ones.obj_vul,3))
print("val_all:", np.round(out_res_MI_ones.val_all,3))
print("val_vul:", np.round(out_res_MI_ones.val_vul,3))

print("expOracle_ones")
print("obj_all:", np.round(out_res_expOracle_ones.obj_all,3))
print("obj_vul:", np.round(out_res_expOracle_ones.obj_vul,3))
print("val_all:", np.round(out_res_expOracle_ones.val_all,3))
print("val_vul:", np.round(out_res_expOracle_ones.val_vul,3))
print("num_vul:", out_res_expOracle.is_vul[np.where(out_res_expOracle.is_vul==1)].shape[0])
print("num_trt:", out_res_expOracle.A_pred[np.where(out_res_expOracle_ones.A_pred=="yes")].shape[0])


print("rise_ones")
print("obj_all:", np.round(out_res_rise_ones.obj_all,3))
print("obj_vul:", np.round(out_res_rise_ones.obj_vul,3))
print("val_all:", np.round(out_res_rise_ones.val_all,3))
print("val_vul:", np.round(out_res_rise_ones.val_vul,3))
print("num_vul:", out_res_rise.is_vul[np.where(out_res_rise.is_vul==1)].shape[0])
print("num_trt:", out_res_rise.A_pred[np.where(out_res_rise_ones.A_pred=="yes")].shape[0])


print("base")
print("obj_all:", np.round(out_res_base.obj_all,3))
print("obj_vul:", np.round(out_res_base.obj_vul,3))
print("val_all:", np.round(out_res_base.val_all,3))
print("val_vul:", np.round(out_res_base.val_vul,3))
print("num_vul:", out_res_base.is_vul[np.where(out_res_base.is_vul==1)].shape[0])
print("num_trt:", out_res_base.A_pred[np.where(out_res_base.A_pred=="yes")].shape[0])


print("exp")
print("obj_all:", np.round(out_res_exp.obj_all,3))
print("obj_vul:", np.round(out_res_exp.obj_vul,3))
print("val_all:", np.round(out_res_exp.val_all,3))
print("val_vul:", np.round(out_res_exp.val_vul,3))
print("num_vul:", out_res_exp.is_vul[np.where(out_res_exp.is_vul==1)].shape[0])
print("num_trt:", out_res_exp.A_pred[np.where(out_res_exp.A_pred=="yes")].shape[0])

print("MI")
print("obj_all:", np.round(out_res_MI.obj_all,3))
print("obj_vul:", np.round(out_res_MI.obj_vul,3))
print("val_all:", np.round(out_res_MI.val_all,3))
print("val_vul:", np.round(out_res_MI.val_vul,3))

print("expOracle")
print("obj_all:", np.round(out_res_expOracle.obj_all,3))
print("obj_vul:", np.round(out_res_expOracle.obj_vul,3))
print("val_all:", np.round(out_res_expOracle.val_all,3))
print("val_vul:", np.round(out_res_expOracle.val_vul,3))
print("num_vul:", out_res_expOracle.is_vul[np.where(out_res_expOracle.is_vul==1)].shape[0])
print("num_trt:", out_res_expOracle.A_pred[np.where(out_res_expOracle.A_pred=="yes")].shape[0])


print("rise")
print("obj_all:", np.round(out_res_rise.obj_all,3))
print("obj_vul:", np.round(out_res_rise.obj_vul,3))
print("val_all:", np.round(out_res_rise.val_all,3))
print("val_vul:", np.round(out_res_rise.val_vul,3))
print("num_vul:", out_res_rise.is_vul[np.where(out_res_rise.is_vul==1)].shape[0])
print("num_trt:", out_res_rise.A_pred[np.where(out_res_rise.A_pred=="yes")].shape[0])


In [ ]:

def optimal_AS_inf(df, proj, is_sim, s_type, X_names, XS_names, y_fn,
                   is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, fit_qua0, fit_qua1, fit_qua0_pqa, fit_qua1_pqa):
    """
    Used for real data with disc S | simulation with disc/cont S

    get df with minority index
        input:  df_train for E(Y|X,S,A) and df_test for Yhat -> optimal S
        output: df_test with vulnerable index
    """
    df_use = df.copy()

    S_names = list( set(XS_names) - set(X_names) )
    # S grid from empirical test data

    X_df = df_use[X_names].copy()

    # link S* to observed data
    XS_df = df_use[X_names+S_names].copy()


    def invlogit(x):
        return 1 / (1 + np.exp(-x))
    def logit(p):
        return np.log(p / (1 - p))


    XS_df["EY_0"] = fit_Y_XS_A0.predict_proba(df_test[XS_names])[:,1]
    XS_df["EY_1"] = fit_Y_XS_A1.predict_proba(df_test[XS_names])[:,1]

    XS_df["QY_0"] = invlogit(fit_qua0.predict(df_test[X_names]))
    XS_df["QY_1"] = invlogit(fit_qua1.predict(df_test[X_names]))

    XS_df["QY_0_pqa"] = invlogit(fit_qua0_pqa.predict(df_test[X_names]))
    XS_df["QY_1_pqa"] = invlogit(fit_qua1_pqa.predict(df_test[X_names]))



    XS_df['M_opt']=0
    XS_df['M_opt'] = np.where((((XS_df['QY_0']>XS_df['QY_1'])&(XS_df['QY_1_pqa']>=XS_df['EY_1']))|((XS_df['QY_1']>XS_df['QY_0'])&(XS_df['QY_0_pqa']>=XS_df['EY_0']))),1,0)

    merge_df=XS_df.copy()

    # minor_index: 1-vulnerable, 0-rest
    print("M_opt 1:vulnerable 0:rest\n", merge_df['M_opt'].value_counts(dropna=False))
    M_opt = merge_df['M_opt'].copy()
    assert( M_opt.shape[0] == X_df.shape[0] )
    print(XS_df)
    return M_opt

plots

In [ ]:

!pip install shap

import shap


def plot_shap(out, save_path):
    tf.compat.v1.disable_v2_behavior()

    _, _, model = class_pred(out.df, out.df_test, out.X_names, out.y_qua, out.w_qua, out.s_type, out.is_tune,None, transfer_weights=out.transfer_weights, S_names=["S"])

    explainer = shap.DeepExplainer(model, out.df[out.X_names])
    shap_values = explainer.shap_values(out.df_test[out.X_names].values)
    shap_abs(shap_values[0], out.df_test[out.X_names], save_path+"fig_rise")

def plot_shap(model,shap_names,df_test,title): #df_test[X_names]
    tf.compat.v1.disable_v2_behavior()

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(df_test[shap_names])

    shap_abs(shap_values, df_test[shap_names],title)



def shap_abs(df_shap, df_in,title):
    """
    Simplified version of SHAP representation for easier interpretation
    modified based on https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d
    """

    shap_v = pd.DataFrame(df_shap)
    feature_list = df_in.columns
    shap_v.columns = feature_list
    df_v = df_in.copy().reset_index().drop('index',axis=1)

    # Determine the correlation in order to plot with different colors
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    corr_df.columns  = ['Variable','Corr']

    corr_df['Sign'] = np.where(corr_df['Corr']>0,'dimgrey','lightgrey')

    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(5,6),legend=False, title=title)
    ax.set_xlabel("SHAP Value (Darkgrey = Positive Impact)")
    fig = ax.get_figure()
    fig.tight_layout()





In [ ]:
import cvxpy as cp
import numpy as np

def entr(w):
    # Define the entropy function
    return cp.sum(cp.entr(w))

def entr_balance(tol, x_in_rct, x_in_rwe):
    # the optimization problem for the nonparametric weighting method
    n = x_in_rct.shape[0]
    w = cp.Variable(n)
    p = x_in_rct.shape[1]-1
    objective = cp.Maximize(cp.sum(cp.entr(w)))
    constraints = [w >= 0, cp.sum(w) == 1]
    for k in range(p):
        constraints += [cp.abs(cp.sum(w*x_in_rct[:,k+1]) - np.mean(x_in_rwe[:,k+1])) <= tol*np.std(x_in_rwe[:,k+1])*np.sqrt(n/(n-1))]
        constraints += [cp.abs(cp.sum(w*(x_in_rct[:,k+1]**2)) - np.mean(x_in_rwe[:,k+1]**2)) <= tol*np.std(x_in_rwe[:,k+1]**2)*np.sqrt(n/(n-1))]

        print("check std:",np.std(x_in_rwe[:,k+1]))

    prob = cp.Problem(objective, constraints)
    w_tilde = np.zeros(n)
    try:
        result = prob.solve(solver=cp.ECOS)
        w_tilde = w.value

        print(k,w_tilde)

        if w_tilde is None:
            w_tilde = np.zeros(n)
        else:
            if np.isnan(w_tilde[0]):
                w_tilde = np.zeros(n)

    except cp.error.SolverError as e:

        pass
    return w_tilde


def weight_tuned_entr(tols, x_in_rct, x_in_rwe, prop=0.5, smps=10):
    n = x_in_rct.shape[0]
    sds = np.apply_along_axis(np.std, 0, x_in_rwe)
    means = np.apply_along_axis(np.mean, 0, x_in_rwe)
    sds2 = np.apply_along_axis(np.std, 0, x_in_rwe**2)
    moment2 = np.apply_along_axis(np.mean, 0, x_in_rwe**2)
    cov_diff_bars = []
    w_tildes = np.full((n, len(tols)), np.nan)
    for i in range(len(tols)):
        tol = tols[i]
        w_tilde = entr_balance(tol, x_in_rct, x_in_rwe)
        w_tilde[w_tilde < 0] = 0  # the optimization results sometimes return negative weights, so clip them to 0
        if np.sum(w_tilde) == 0:
            cov_diff_bar = 1e+10
        else:
            w_tildes[:, i] = w_tilde
            cov_diffs = []
            for s in range(smps):
                boot_ind = np.random.choice(n, size=int(round(prop * n)), replace=True)
                x_in_rct_boot = x_in_rct[boot_ind, :]
                w_tilde_boot = w_tilde[boot_ind]
                cov_diff = (np.sum(x_in_rct_boot * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - means)[1:] / sds[1:]
                cov_diff_moment2 = (np.sum(x_in_rct_boot**2 * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - moment2)[1:] / sds2[1:]

                cov_diffs.append(np.sum(np.concatenate((cov_diff, cov_diff_moment2))**2))  # L2 measure
            cov_diff_bar = np.mean(cov_diffs)
        cov_diff_bars.append(cov_diff_bar)
    tuned_tol = tols[np.argmin(cov_diff_bars)]
    if min(cov_diff_bars) < 1e+10:
        return w_tildes[:, np.argmin(cov_diff_bars)]
    else:
        raise ValueError('************ optimization not feasible *****************')


def learn_weights(y_in_rct, x_in_rct, a_in_rct, x_in_rwe, w_method, misspecify, N):
    # w_method: 1, 2, 3, 4
    n = x_in_rct.shape[0]
    p = x_in_rct.shape[1] - 1
    if w_method == 4:

        w_tilde = weight_tuned_entr([ 1.000], x_in_rct, x_in_rwe)
    else:
        raise ValueError("Wrong input for weighting method")
    return w_tilde



In [ ]:

def set_random(seed_value):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

    return


def normalize(df_tr, df_te, names_to_norm):
    df = df_tr.copy()
    df_test = df_te.copy()

    # normalize X & S
    scaler = StandardScaler()
    df[names_to_norm] = scaler.fit_transform(df[names_to_norm])
    df_test[names_to_norm] = scaler.transform(df_test[names_to_norm])

    return(df.copy(), df_test.copy())


def get_train_test(df_all, s_type, seed_value):
    # train-test split by A
    df0_ori = df_all[df_all["A"]==0].reset_index(drop=True).copy()
    df1_ori = df_all[df_all["A"]==1].reset_index(drop=True).copy()
    df0_train, df0_test = train_test_split(df0_ori, test_size=0.2, random_state=seed_value)
    df1_train, df1_test = train_test_split(df1_ori, test_size=0.2, random_state=seed_value)

    df_ori = pd.concat([df0_train, df1_train]).reset_index(drop=True).copy()
    df_test_ori = pd.concat([df0_test, df1_test]).reset_index(drop=True).copy()


    print("df_ori.shape", df_ori.shape)
    if (s_type == "disc"):
        print("train Pr(S=0)/Pr(S=1):", round(sum(df_ori["S"]==0) / sum(df_ori["S"]==1),3))
    print("train Pr(A=0)/Pr(A=1):", round(sum(df_ori["A"]==0) / sum(df_ori["A"]==1),3))

    print("df_test_ori.shape", df_test_ori.shape)
    if (s_type == "disc"):
        print("test Pr(S=0)/Pr(S=1):", round(sum(df_test_ori["S"]==0) / sum(df_test_ori["S"]==1),3))
    print("test Pr(A=0)/Pr(A=1):", round(sum(df_test_ori["A"]==0) / sum(df_test_ori["A"]==1),3))

    return(df_ori.copy(), df_test_ori.copy())






def get_ps_fit(df_train, use_covars, transfer_weights):
    df_tr = df_train.copy()

    regr = RandomForestClassifier(random_state=1234).fit(df_tr[use_covars], df_tr["A"],sample_weight=transfer_weights)

    return(regr)

def keras_wrap(x_train, train_labels, train_wts, x_test, loss_fn, act_out,
               layer=2, node=512, dropout=0.2, n_epoch=100, bsize=64, act="relu",
               opt="Adam", val_split=0.2, is_early_stop=True, verb=0, obj=None):


    val_split=None

    x_train=x_train.reset_index(drop=True)
    if is_early_stop:
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
        callback = [early_stop]
    else:
        callback = None

    # set input_dim for the number of features
    if len(x_train.shape) == 1:
        input_dim = 1
    else:
        input_dim = x_train.shape[1]

    # model
    if obj=="quan":

        param_grid = {'n_estimators': [100, 200, 300],
                      'learning_rate': [0.1, 0.2, 0.3],
                      'max_depth': [3, 4, 5]
                      }
        param_grid = {
            'n_estimators': [200],
            'max_depth': [3, 4],
            'learning_rate': [0.2, 0.3],
            'subsample': [0.5, 0.75, 1],
            'colsample_bytree': [0.6, 0.8],
            'gamma': [0]
            }

# Create an instance of the XGBRegressor model
        model = XGBRegressor(objective=loss_fn, seed=5292023)

# Perform grid search with cross-validation
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
        grid_search.fit(x_train, train_labels, sample_weight=train_wts)

# Get the best model and its hyperparameters
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

# Use the best model for prediction
        model = best_model
        pred_test = model.predict(x_test).flatten()
        pred_train = model.predict(x_train).flatten()


    else:
        param_grid = {
            'n_estimators': [150],
            'max_depth': [1, 2, 3],
            'learning_rate': [0.3, 0.4],
            'subsample': [0.5, 0.75, 1],
            'colsample_bytree': [0.6, 0.8],
            'gamma': [0]
            }

    # Create an XGBoost classifier
        model = XGBClassifier(seed=5292023)

    # Perform grid search with cross-validation
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='roc_auc')
        grid_search.fit(x_train, train_labels, sample_weight=train_wts)

    # Get the best model and its hyperparameters
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        model_caret = best_model



        pred_test = model_caret.predict_proba(x_test)[:, 1].flatten()
        pred_train = model_caret.predict_proba(x_train)[:, 1].flatten()
        model=model_caret

        print(pred_test.shape)
        print("check here (pred_test in keras wrap)")
        print(pred_test)


    return pred_test, pred_train, model


def hyper_tuning(x_train, train_labels, train_wts, loss_fn, act_out,
                 param_grid, n_cv=5, n_jobs=1):

    # set input_dim for the number of features
    if len(x_train.shape) == 1:
        input_dim = 1
    else:
        input_dim = x_train.shape[1]

    def create_model(layers,nodes,activation,optimizer,dropout):
        model = Sequential()
        for i in range(layers):
            if i==0:
                model.add(Dense(nodes, input_dim=input_dim))
                model.add(Activation(activation))
                model.add(Dropout(dropout))
            else:
                model.add(Dense(nodes, activation=activation))
                model.add(Activation(activation))
                model.add(Dropout(dropout))

        model.add(Dense(units=1, activation=act_out))

        model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'],weighted_metrics=['acc'])
        return model

    model = KerasRegressor(build_fn=create_model, verbose=0)

    assert param_grid is not None
    layers = param_grid['layers']
    nodes = param_grid['nodes']
    dropout = param_grid['dropouts']
    activation = param_grid['acts']
    optimizer = param_grid['opts']
    bsizes = param_grid['bsizes']
    n_epochs = param_grid['n_epochs']

    param_grid = dict(layers=layers, nodes=nodes, activation=activation, optimizer=optimizer,
                      dropout=dropout, batch_size=bsizes, epochs=n_epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=n_cv, n_jobs=n_jobs)

    grid_result = grid.fit(x_train, train_labels, sample_weight=train_wts)

    print("hyperparams tuning:", grid_result.best_score_,grid_result.best_params_)

    bst_params = grid_result.best_params_
    layer = bst_params['layers']
    node = bst_params['nodes']
    dropout = bst_params['dropout']
    n_epoch = bst_params['epochs']
    bsize = bst_params['batch_size']
    act = bst_params['activation']
    opt = bst_params['optimizer']

    return layer, node, dropout, n_epoch, bsize, act, opt





def fit_expectation(obj_name, df, use_covars, df_pred, is_class, is_tune, param_grid, transfer_weights, df_val=None):
    """E(Y|X=x,A=a) or E(Y|X=x,S=s,A=a)
    (model separated by A)
    """

    df_tr = df.copy()
    df_te = df_pred.copy()
    if df_val is not None:
        df_va = df_val.copy()

    # fit on df0/df1
    if is_class:
        loss_fn = 'binary_crossentropy'
        act_out = 'sigmoid'
    else:
        loss_fn = "mean_squared_error"
        act_out = None


    Yhat, _, regr = keras_wrap(df_tr[use_covars], df_tr["Y"], transfer_weights,
                        df_te[use_covars], loss_fn, act_out,
                        layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu", opt="Adam",
                        val_split=0.2, is_early_stop=True, verb=0)

    if df_val is not None:
        if not is_class:
            y_tr = regr.predict(df_tr[use_covars])
            rms = mean_squared_error(df_tr['Y'], y_tr, squared=False)
            met = rms / (np.max(df_tr['Y']) - np.min(df_tr['Y']))
            print(obj_name, "evaluate on train: NRMSE", met)

            y_va = regr.predict(df_va[use_covars])
            rms = mean_squared_error(df_va['Y'], y_va, squared=False)
            met = rms / (np.max(df_va['Y']) - np.min(df_va['Y']))
            print(obj_name, "evaluate on test : NRMSE", met)
        elif is_class:
            y_tr = regr.predict(df_tr[use_covars])
            met = roc_auc_score(df_tr['Y'], y_tr)
            print(obj_name, "evaluate on train: AUC", met)

            y_va = regr.predict(df_va[use_covars])
            met = roc_auc_score(df_va['Y'], y_va)
            print(obj_name, "evaluate on test : AUC", met)

    return(Yhat, regr)


def fit_quantile(x_train, train_labels, x_test, q, is_tune, param_grid, transfer_weights):
    """
    quantile regression: yhat | X,A
    """

    def invlogit(x):
        return 1 / (1 + np.exp(-x))
    def logit(p):
        return np.log(p / (1 - p))
    train_labels=logit(train_labels)

    def tilted_loss(q, y, f):
        """quantile loss for Keras"""

        e = (y - f)
        return keras.backend.mean(keras.backend.maximum(q * e, (q - 1) * e), axis=-1)

    act_out = None

    act_out = 'sigmoid'
    def loss_fn(y, f):

        alpha = q
        x = y - f

        grad = np.where(x >= 0, -alpha,  1 - alpha)
        hess = np.ones_like(x)
        return grad, hess

    if is_tune:
        layer, node, dropout, n_epoch, bsize, act, opt = \
                    hyper_tuning(x_train, train_labels, transfer_weights, loss_fn, act_out,
                        param_grid, n_cv=5, n_jobs=1)

        pred_test, _, model = keras_wrap(x_train, train_labels, transfer_weights,
                        x_test, loss_fn, act_out,
                        layer, node, dropout, n_epoch, bsize, act, opt,
                        val_split=None, is_early_stop=False, verb=0, obj="quan")
    else:


        pred_test, _, model = keras_wrap(x_train, train_labels, transfer_weights, x_test, loss_fn, act_out,
                layer=2, node=512, dropout=0.2, n_epoch=100, bsize=32, act="relu",
                opt="Adam", val_split=0.2, is_early_stop=True, verb=0, obj="quan")

    train_labels=invlogit(train_labels)
    pred_test=invlogit(pred_test)
    return model, pred_test




def fit_infinite(df, X_names, XS_names, fit):
    df_use = df.copy()

    S_names = list( set(XS_names) - set(X_names) )

    if len(S_names) == 1:
        s_min = int(np.min(df_use[S_names[0]]))
        s_max = int(np.max(df_use[S_names[0]]))
        s_grid = range(s_min, s_max+1)
    else:
        s_lst = []
        for s_var in S_names:
            s_min = int(np.min(df_use[s_var]))
            s_max = int(np.max(df_use[s_var]))
            s_grid = range(s_min, s_max+1)
            s_lst.append(np.array(s_grid))
        s_grid = s_lst.copy()
        print("s_grid:", s_grid)

    X_df = df_use[X_names].copy()
    idx = range(X_df.shape[0])
    X_df["idx"] = idx

    if len(S_names) == 1:
        idx_s = np.array(np.meshgrid(idx, s_grid)).reshape(2, len(idx)*len(s_grid)).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx',S_names[0]])
    else:
        idx_s_lst = [np.array(idx)] + s_grid
        idx_s = np.array(np.meshgrid(*np.array(idx_s_lst, dtype=object))).reshape(1+len(S_names), -1).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx']+S_names)

    aug_df = X_df.merge(idx_s, on='idx', how='left').copy()

    # predict Y
    aug_df["pred"] = fit.predict(aug_df[XS_names])

    smry = aug_df.groupby(by="idx", as_index=False).agg({"pred":["min"]})

    return(smry.loc[:,("pred","min")].values.copy())



def get_label_wt(g1, g0):
    """get label & weight for classification"""
    c1 = g1
    c2 = g0
    label = np.sign(c1 - c2)
    label = np.where(label==1, 1, 0)

    weight = np.abs(c1 - c2)


    return(label, weight)




def class_pred(df_train, df_test, use_covars, label, weight, s_type, is_tune, param_grid,transfer_weights,S_names=None):
    """binary classification with sample weights for decision rule by Keras
    """

    df_tr = df_train.reset_index(drop=True).copy()
    df_te = df_test.reset_index(drop=True).copy()
    print(df_train)
    print(df_test)
    print(use_covars)
    print(label)

    loss_fn = 'binary_crossentropy'
    act_out = 'sigmoid'



    prob_test, prob_train, model = keras_wrap(df_tr[use_covars], label, weight*transfer_weights,
                    df_te[use_covars], loss_fn, act_out,
                    layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu",
                    opt="Adam", val_split=None, is_early_stop=True, verb=0)


    pred_test = np.where(prob_test>0.5, "yes", "no")
    pred_train = np.where(prob_train>0.5, "yes", "no")

    # print table
    print("pred_train table:", collections.Counter(pred_train.ravel()))
    print("pred_test table:", collections.Counter(pred_test.ravel()))
    if s_type == "disc": #disc S
        df_tr["pred_train"] = pred_train
        df_te["pred_test"] = pred_test
        print(pd.crosstab(df_tr[S_names[0]], df_tr["pred_train"], normalize='index'))
        print(pd.crosstab(df_te[S_names[0]], df_te["pred_test"], normalize='index'))
    print("="*30)

    return(pred_test, pred_train, model)


def get_A_names(is_base, is_exp, is_expOracle, is_qua, is_inf):
    A_names = list()
    A_names.append("A_obs")
    if is_base:
        A_names.append("A_base")
    if is_exp:
        A_names.append("A_exp")
    if is_expOracle:
        A_names.append("A_expOracle")
    if is_qua or is_inf:
        A_names.append("A_qua")
    print("A_names:",A_names)

    return A_names


def add_Apred_Midx(df, A_names, A_base, A_exp, A_expOracle, A_qua):
    df_use = df.copy()

    # add A_pred to df
    df_use["A_obs"] = np.where(df_use["A"]==1, "yes","no")
    if "A_base" in A_names:
        df_use["A_base"] = A_base
    if "A_exp" in A_names:
        df_use["A_exp"] = A_exp
    if "A_expOracle" in A_names:
        df_use["A_expOracle"] = A_expOracle
    if "A_qua" in A_names:
        df_use["A_qua"] = A_qua

    return df_use.copy()



def minor_rule(df, use_covars, s_type, S_name):
    """use decision tree to print out rule of vulnerable group (X -> S)
        only apply when S is discrete
    """
    df_use = df.copy()

    df_use["S_minor"] = np.where(np.isnan(df_use[S_name]), -999,df_use[S_name])
    print('df_use["S_minor"]', df_use["S_minor"].value_counts(dropna=False))

    if s_type == "disc":
        clf = DecisionTreeClassifier(max_depth=3, random_state=4211)
    else:
        clf = DecisionTreeRegressor(max_depth=3, random_state=4211)

    clf.fit(df_use[use_covars], df_use["S_minor"])


    tree_rules = export_text(clf, feature_names = list(use_covars))
    print(S_name)
    print(tree_rules)

    clf.predict(df_use[use_covars])

    return





def expect_reward(df, A_name, minor_name):
    df_use = df.copy()

    df_use["A_obs"]=np.where(df_use["A"]==1, "yes","no")

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    numer = df_use["Y"] * (df_use["A_obs"] == df_use[A_name]) / df_use["ps"]
    denom = (df_use["A_obs"] == df_use[A_name]) / df_use["ps"]
    reward_all = np.sum(numer) / np.sum(denom)

    if not minor_df.empty:
        numer = minor_df["Y"] * (minor_df["A_obs"] == minor_df[A_name]) / minor_df["ps"]
        denom = (minor_df["A_obs"] == minor_df[A_name]) / minor_df["ps"]
        reward_minor = np.sum(numer) / np.sum(denom)
    else:
        reward_minor = np.nan

    if not rest_df.empty:
        numer = rest_df["Y"] * (rest_df["A_obs"] == rest_df[A_name]) / rest_df["ps"]
        denom = (rest_df["A_obs"] == rest_df[A_name]) / rest_df["ps"]
        reward_rest = np.sum(numer) / np.sum(denom)
    else:
        reward_rest = np.nan

    print(A_name, "reward  ",
          "all:",round(reward_all,3),
          "minor:",round(reward_minor,3),
          "rest:",round(reward_rest,3))

    return reward_all, reward_minor, reward_rest


def get_pred_Y(df, A_name, XS_names, y_fn, model_sxa0, model_sxa1):
    """generate Y based on predicted A (treatment assignment)
    model: Y|X,S,A=a
    should use unnormalized X & S for simulation & normalized for real data!
    """
    df_use = df[XS_names].copy()

    df_use["A"] = np.where(df[A_name]=="yes", 1,0)
    if y_fn is not None:
        Y_pred = df_use.eval(y_fn)
    else:

        Y_pred0 = model_sxa0.predict_proba(df_use[XS_names])[:,1].flatten()
        Y_pred1 = model_sxa1.predict_proba(df_use[XS_names])[:,1].flatten()
        Y_pred = np.where(df_use["A"]==0, Y_pred0, Y_pred1)
    assert len(Y_pred.shape)==1

    return Y_pred


def get_value_fn(df, A_name, XS_names, minor_name, y_fn, model_sxa0, model_sxa1):
    """get value function for subgroup of interest"""
    df_use = df.copy()

    df_use["Y_pred"] = get_pred_Y(df_use, A_name, XS_names, y_fn, model_sxa0, model_sxa1)

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    value_all = np.mean(df_use["Y_pred"].values)

    if not minor_df.empty:
        value_minor = np.mean(minor_df["Y_pred"].values)
        minorPct=minor_df.shape[0]/df_use.shape[0]
    else:
        value_minor = np.nan
        minorPct=np.nan

    if not rest_df.empty:
        value_rest = np.mean(rest_df["Y_pred"].values)
    else:
        value_rest = np.nan

    print(A_name, "value   ",
          "all:",round(value_all,3),
          "minor:",round(value_minor,3),
          "rest:",round(value_rest,3),
          "minorPct:",round(minorPct,3))

    return value_all, value_minor, value_rest, minorPct



def get_obj_qua(df, X_names, s_type, A_name, minor_name, fit_qua0, fit_qua1):
    """
    Used for real data with cont S (no need S grid)

    df includes X,A_test (A_test is from prediction)
    objective: E[Gs{E[Y|X,S,A=d(x)]}]
    -> Quantile_regress Y|X,A -> E[.]
    -> take avg over all subjects
    """
    df_use = df.copy()
    del df_use['A']
    del df_use['Y']

    if s_type != "cont":
        assert(1 == 0)


    Y_pred0 = 1 / (1 + np.exp(-fit_qua0.predict(df_use[X_names]))).flatten()
    Y_pred1 = 1 / (1 + np.exp(-fit_qua1.predict(df_use[X_names]))).flatten()

    df_use["A"] = np.where(df_use[A_name]=="yes", 1,0) #contains X & A
    df_use["Y"] = np.where(df_use["A"]==0, Y_pred0, Y_pred1)

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    obj_all = np.mean(df_use["Y"].values)

    if not minor_df.empty:
        obj_minor = np.mean(minor_df["Y"].values)
    else:
        obj_minor = np.nan

    if not rest_df.empty:
        obj_rest = np.mean(rest_df["Y"].values)
    else:
        obj_rest = np.nan

    print(A_name, "obj     ",
          "all:",round(obj_all,3),
          "minor:",round(obj_minor,3),
          "rest:",round(obj_rest,3))

    return obj_all, obj_minor, obj_rest


def get_obj_inf(df, X_names, XS_names, A_name, minor_name, y_fn, s_type, is_class, qua_use, model_sxa0, model_sxa1):
    """
    Used for real data with disc S | simulation with disc/cont S
    df includes X,A_test (A_test is from prediction)
    objective: E[Gs{E[Y|X,S,A=d(x)]}]
    -> get E[Y|X,S,A=d(x)] from true distn (simulation) or fitted model (real data)
    -> given X & A, generate S from empirical test data
    -> find quan/inf wrt S
    -> take avg over all subjects
    """
    df_use = df.copy()

    # S grid from empirical test data
    if s_type == "disc": # real/simulation with disc S
        S_names = list( set(XS_names) - set(X_names) )
        if len(S_names) == 1:
            s_min = int(np.min(df_use["S"]))
            s_max = int(np.max(df_use["S"]))
            s_grid = range(s_min, s_max+1)
            if y_fn is None:
                s_grid = [s_grid]
        else:
            s_lst = []
            for s_var in S_names:
                s_min = int(np.min(df_use[s_var]))
                s_max = int(np.max(df_use[s_var]))
                s_grid = range(s_min, s_max+1)
                s_lst.append(np.array(s_grid))
            s_grid = s_lst.copy()
    elif s_type == "cont" and y_fn is not None: # simulation with cont S
        s_grid = df["S"].values.copy()
        assert len(s_grid) > 2
    else:
        assert(1 == 0)

    # augment X&A with S grid
    df_XA = df_use[[minor_name]+X_names].copy() #contains X
    df_XA["A"] = np.where(df_use[A_name]=="yes", 1,0) #contains X & A
    idx = range(df_XA.shape[0])
    df_XA["idx"] = idx

    if y_fn is None:
        idx_s_lst = [np.array(idx)] + s_grid
        idx_s = np.array(np.meshgrid(*np.array(idx_s_lst, dtype=object))).reshape(1+len(S_names), -1).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx']+S_names)
    else: # if remove, may occur error in .eval(y_fn) # this works as long as there is no multi-S in simulation
        idx_s = np.array(np.meshgrid(idx, s_grid)).reshape(2, len(idx)*len(s_grid)).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx','S'])

    aug_df = df_XA.merge(idx_s, on='idx', how='left').reset_index(drop=True).copy()

    # get E[Y|X,S,A=d(x)]
    if y_fn is not None: # from true distn (simulation)
        aug_df["Y"] = aug_df.eval(y_fn)
    else: # from fitted model (real data)

        Y_pred0 = model_sxa0(aug_df[XS_names]).flatten()
        Y_pred1 = model_sxa1(aug_df[XS_names]).flatten()
        aug_df["Y"] = np.where(aug_df["A"]==0, Y_pred0, Y_pred1)

    # find quantile/inf wrt S
    def quan(x):
        return x.quantile(qua_use)

    minor_df = aug_df[aug_df[minor_name]==1].reset_index(drop=True).copy()
    rest_df = aug_df[aug_df[minor_name]==0].reset_index(drop=True).copy()

    # Gs{E[Y|X,S,A=d(x)]}
    if (s_type == "cont"):
        agg_fn = {"Y": quan}
    elif (s_type == "disc"):
        agg_fn = {"Y": "min"}

    smry_all = aug_df.groupby(by="idx", as_index=False).agg(agg_fn)
    smry_minor = minor_df.groupby(by="idx", as_index=False).agg(agg_fn)
    smry_rest = rest_df.groupby(by="idx", as_index=False).agg(agg_fn)

    # obj: E{G(.)}
    obj_all = np.mean(smry_all.loc[:,"Y"].values)

    if not minor_df.empty:
        obj_minor = np.mean(smry_minor.loc[:,"Y"].values)
    else:
        obj_minor = np.nan

    if not rest_df.empty:
        obj_rest = np.mean(smry_rest.loc[:,"Y"].values)
    else:
        obj_rest = np.nan

    print(A_name, "obj     ",
          "all:",round(obj_all,3),
          "minor:",round(obj_minor,3),
          "rest:",round(obj_rest,3))

    return obj_all, obj_minor, obj_rest



def dict_mean(dict_list):
    mean_list = list()
    mean_dict = dict()
    for A_name in dict_list[0].keys():
        mean_dict[A_name] = dict()
        for metric in dict_list[0][A_name].keys():
            mean_dict[A_name][metric] = np.nansum([d[A_name][metric] for d in dict_list]) / len(dict_list)
            mean_list.append(mean_dict[A_name][metric])

    mean_list = np.array(mean_list)
    mean_list = np.reshape(mean_list, (len(dict_list[0].keys()), -1)).T
    mean_df = pd.DataFrame(data=mean_list, columns=dict_list[0].keys())
    mean_df.index = list(dict_list[0][list(dict_list[0].keys())[0]].keys())

    return mean_dict, mean_df


def dict_std(dict_list):
    """
    Replicate this sampling test data procedure 100 times,
    estimate values of each ITR for each replicate,
    and get the averaged estimated values (standard error)
    """
    mean_list = list()
    mean_dict = dict()
    for A_name in dict_list[0].keys():
        mean_dict[A_name] = dict()
        for metric in dict_list[0][A_name].keys():
            mean = np.nansum([d[A_name][metric] for d in dict_list]) / len(dict_list)
            variance = np.nansum([(d[A_name][metric] - mean)**2 for d in dict_list]) / len(dict_list)
            mean_dict[A_name][metric] = variance**0.5 / (len(dict_list))**0.5  #SE
            mean_list.append(mean_dict[A_name][metric])

    mean_list = np.array(mean_list)
    mean_list = np.reshape(mean_list, (len(dict_list[0].keys()), -1)).T
    mean_df = pd.DataFrame(data=mean_list, columns=dict_list[0].keys())
    mean_df.index = list(dict_list[0][list(dict_list[0].keys())[0]].keys())

    return mean_dict, mean_df


def prop_treat(df, A_name, minor_name):
    df_use = df.reset_index(drop=True).copy()

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    prop_all = df_use[df_use[A_name]=="yes"].shape[0] / df_use.shape[0]

    if not minor_df.empty:
        prop_minor = minor_df[minor_df[A_name]=="yes"].shape[0] / minor_df.shape[0]
    else:
        prop_minor = np.nan

    if not rest_df.empty:
        prop_rest = rest_df[rest_df[A_name]=="yes"].shape[0] / rest_df.shape[0]
    else:
        prop_rest = np.nan

    print(A_name, "pr_treat",
          "all:",round(prop_all,3),
          "minor:",round(prop_minor,3),
          "rest:",round(prop_rest,3))

    return prop_all, prop_minor, prop_rest




def eval_metrics(minor_name, res_df, A_names, X_names, XS_names, y_fn, s_type, is_class,
                    qua_use, fit_Y_XS_A0, fit_Y_XS_A1, fit_qua0, fit_qua1, res_df2):
    print(minor_name)
    print("="*30)
    res = dict()
    for A_name in A_names:
        res[A_name] = dict()
        # metrics
        prop_all, prop_minor, prop_rest = prop_treat(res_df, A_name, minor_name)
        reward_all, reward_minor, reward_rest = expect_reward(res_df, A_name, minor_name)
        value_all, value_minor, value_rest, minorPct = get_value_fn(res_df, A_name, XS_names, minor_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

        if s_type == "disc" or y_fn is not None: # real with disc S | sim with disc/cont S
            obj_all, obj_minor, obj_rest = get_obj_inf(res_df, X_names, XS_names, A_name, minor_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)
        elif s_type == "cont": # real with cont S
            obj_all, obj_minor, obj_rest = get_obj_qua(res_df, X_names, s_type, A_name, minor_name, fit_qua0, fit_qua1)
        else:
            assert(1 == 0)

        minor_df_train = res_df2[res_df2[minor_name]==1].reset_index(drop=True).copy()
        if not minor_df_train.empty:
            minorPct_train=minor_df_train.shape[0]/res_df2.shape[0]
        else:
            minorPct_train=np.nan

        # save
        res[A_name]["prop_all"] = prop_all;     res[A_name]["prop_minor"] = prop_minor;     res[A_name]["prop_rest"] = prop_rest
        res[A_name]["reward_all"] = reward_all; res[A_name]["reward_minor"] = reward_minor; res[A_name]["reward_rest"] = reward_rest
        res[A_name]["value_all"] = value_all;   res[A_name]["value_minor"] = value_minor;   res[A_name]["value_rest"] = value_rest
        res[A_name]["minorPct_test"] = minorPct; res[A_name]["minorPct_train"] = minorPct_train
        res[A_name]["obj_all"] = obj_all;       res[A_name]["obj_minor"] = obj_minor;       res[A_name]["obj_rest"] = obj_rest
        print("="*30)
    print("="*50)
    return res


